In [1]:
import pandas as pd

In [2]:
EPS = 1

In [3]:
result_file = "test_result_w_succ_diff_nodes_rnn_2025_38_03_10_38_44.csv"
df = pd.read_csv(result_file)

df = df.rename(columns={"Dataset": "Graph", "Actual": "Rank"})

df.head()

,Graph,Rank,Predicted
0,graph_powerlaw_cluster_graph_n7,4.0,3.919666
1,graph_powerlaw_cluster_graph_n7,2.0,2.285690
2,graph_powerlaw_cluster_graph_n7,2.0,1.869913
3,graph_powerlaw_cluster_graph_n7,3.0,3.606593
4,graph_powerlaw_cluster_graph_n7,3.0,3.082959


In [4]:
df['Difference'] = abs(df['Rank']-df['Predicted'])
df['Difference_sq'] = df['Difference']**2
# df.head()

In [5]:
df['EPS'] = df['Rank'].apply(lambda x: EPS if x==0.0 else 0.0)
# df

In [6]:
df['RE'] = df['Difference']/(df['Rank']+df['EPS'])
# df.head()

In [7]:
df

,Graph,Rank,Predicted,Difference,Difference_sq,EPS,RE
0,graph_powerlaw_cluster_graph_n7,4.0,3.919666,0.080334,0.006454,0.0,0.020083
1,graph_powerlaw_cluster_graph_n7,2.0,2.285690,0.285690,0.081619,0.0,0.142845
2,graph_powerlaw_cluster_graph_n7,2.0,1.869913,0.130087,0.016923,0.0,0.065043
3,graph_powerlaw_cluster_graph_n7,3.0,3.606593,0.606593,0.367956,0.0,0.202198
4,graph_powerlaw_cluster_graph_n7,3.0,3.082959,0.082959,0.006882,0.0,0.027653
...,...,...,...,...,...,...,...
5275,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
5276,star_graph_n7,3.0,2.246686,0.753314,0.567482,0.0,0.251105
5277,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
5278,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000


In [8]:
df['RE'].sum()/len(df)

0.09099469291035678

In [9]:
def group_by_table(df, groupby):
    values_count = df[groupby].value_counts().reset_index()
    values_count = values_count.rename(columns={"count": "Count"})
    re_sum = df.groupby(groupby)["RE"].sum().reset_index()
    mre_error = df.groupby(groupby)["RE"].mean().reset_index()
    mre_error = mre_error.rename(columns={"RE": "MRE"})
    mse_error = df.groupby(groupby)["Difference_sq"].mean().reset_index()
    mse_error = mse_error.rename(columns={"Difference_sq": "MSE"})

    return values_count, re_sum, mre_error, mse_error

# Group by Rank

In [10]:
groupby = ['Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_rank.csv", index=False)
df_result


,Rank,Count,RE,MRE,MSE
0,2.0,1687,223.447204,0.132452,0.116465
1,3.0,1672,146.324675,0.087515,0.104928
2,1.0,935,77.012081,0.082366,0.011564
3,4.0,684,30.523966,0.044626,0.050630
4,0.0,253,0.046876,0.000185,0.000002
5,5.0,48,2.999918,0.062498,0.139130
6,6.0,1,0.097259,0.097259,0.340536


# Group by Dataset

In [11]:
groupby = ['Graph']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset.csv", index=False)
df_result


,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3907,350.973073,0.089832,0.078238
1,graph_powerlaw_cluster_graph_n7,1350,128.814419,0.095418,0.087021
2,star_graph_n7,23,0.664486,0.028891,0.053216


# Group by Dataset W/O Rank 0

In [12]:
groupby = ['Graph']

df_wo_0 = df[df['Rank']!=0]
values_count, re_sum, mre_error, mse_error = group_by_table(df_wo_0, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_wo_rank0.csv", index=False)
df_result

,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3777,350.947718,0.092917,0.080931
1,graph_powerlaw_cluster_graph_n7,1243,128.792899,0.103615,0.094512
2,star_graph_n7,7,0.664486,0.094927,0.174851


# Group by Dataset and Rank

In [13]:
groupby = ['Graph', 'Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Graph', 'Count'], ascending=[True, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_rank.csv", index=False)
df_result

,Graph,Rank,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n7,2.0,433,63.486733,0.146621,0.147134
1,graph_powerlaw_cluster_graph_n7,1.0,365,29.794597,0.081629,0.011246
2,graph_powerlaw_cluster_graph_n7,3.0,302,29.439941,0.097483,0.137914
3,graph_powerlaw_cluster_graph_n7,4.0,128,5.218073,0.040766,0.048254
4,graph_powerlaw_cluster_graph_n7,0.0,107,0.021521,0.000201,0.000002
5,graph_powerlaw_cluster_graph_n7,5.0,15,0.853554,0.056904,0.122545
6,graph_random_regular_graph_n7_d4,3.0,1367,116.585242,0.085285,0.097447
7,graph_random_regular_graph_n7_d4,2.0,1253,159.903116,0.127616,0.105949
8,graph_random_regular_graph_n7_d4,1.0,570,47.217483,0.082838,0.011768
9,graph_random_regular_graph_n7_d4,4.0,554,25.061623,0.045238,0.050403
